# make a function for missing values 


### import package 

In [51]:
import pandas as pd 
import pathlib
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import time


### importing data

In [52]:
def read_file(file_name : str )  -> pd.DataFrame:
    """
    summary
    """
    try:
        dir_folder = pathlib.Path().cwd().parent
        print(dir_folder)
        file_path  = dir_folder / "data" 
        print(file_path)
        df = pd.read_csv(os.path.join(file_path/file_name))
        return df
    except FileNotFoundError:
        print(f"Error: The file at '{file_name}' was not found.")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        
        
        
df = read_file('E:/machine_learning_project/machine_learning_cour2/ames.csv')      
df.head()

e:\machine_learning_project
e:\machine_learning_project\data


,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,NaN,0,5,2010,WD,Normal,215000,-93.619754,42.054035
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,Minimum_Privacy,NaN,0,6,2010,WD,Normal,105000,-93.619756,42.053014
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.619387,42.052659
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,No_Fence,NaN,0,4,2010,WD,Normal,244000,-93.617320,42.051245
4,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,Minimum_Privacy,NaN,0,3,2010,WD,Normal,189900,-93.638933,42.060899


### data structure

In [53]:
def data_diagnostic(df):
        print("#"*50)
        print(df.info())
        print("#"*50)
        print("The number of total rows  {x: .0f} ".format(x=df.shape[0]))
        print("The number of total variables {x: .0f} ".format(x=df.shape[1]))
        print("The variables names {x:} ".format(x=list(df.columns.values)))

        column_headers =list(df.columns.values)
        qualitative_columns = [col for col in column_headers if df[col].dtype=="object"]
        quantitative_columns = [col for col in column_headers if df[col].dtype in ['int64', 'float64']]

        print("The qualitative variables {x:} ".format(x=qualitative_columns))
        print("The quantitative variables {x:} ".format(x=quantitative_columns))
        print("#"*50)
        print("Total number missing value {x:} ".format(x=df.isnull().sum()))
data_diagnostic(df=df)

##################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MS_SubClass         2930 non-null   object 
 1   MS_Zoning           2930 non-null   object 
 2   Lot_Frontage        2930 non-null   int64  
 3   Lot_Area            2930 non-null   int64  
 4   Street              2930 non-null   object 
 5   Alley               2930 non-null   object 
 6   Lot_Shape           2930 non-null   object 
 7   Land_Contour        2930 non-null   object 
 8   Utilities           2930 non-null   object 
 9   Lot_Config          2930 non-null   object 
 10  Land_Slope          2930 non-null   object 
 11  Neighborhood        2930 non-null   object 
 12  Condition_1         2930 non-null   object 
 13  Condition_2         2930 non-null   object 
 14  Bldg_Type           2930 non-null   object 
 15  Hous

In [54]:
def na(df, percent = True):
    srs = df.isna().sum()[df.isna().sum() > 0]
    srs = srs.sort_values(ascending=False)
    if percent:
        print('% of NaNs in df:')
        return srs / df.shape[0]
    else:
        print('# of NaNs in df:')
        return srs

na(df, False)


# of NaNs in df:


Misc_Feature    2824
Mas_Vnr_Type    1775
dtype: int64

### data cleaning 

#### missing values 

In [55]:
class MissingDataHandler:
    def __init__(self, df):
        self.df = df  # Appliquer les changements directement sur le DataFrame original
    # 1. Supprimer les colonnes avec trop de valeurs manquantes
    def drop_column(self, threshold=0.5):
        """Drop columns with more than threshold% missing values."""
        self.df.dropna(thresh=len(self.df) * (1 - threshold), axis=1, inplace=True)
    # 2. Supprimer les lignes avec des valeurs manquantes
    def drop_row(self):
        """Drop rows with any missing values."""
        self.df.dropna(inplace=True)
    # 3. Imputation avec la moyenne ou la médiane
    def impute_mean_median(self, strategy='mean'):
        """Impute missing values in all numerical columns with mean or median."""
        num_cols = self.df.select_dtypes(include=[np.number]).columns
        for col in num_cols:
            imputer = SimpleImputer(strategy=strategy)
            self.df[col] = imputer.fit_transform(self.df[[col]])
    
    def group_imputation(self, group_by, strategy='mean'):
        """Group-wise imputation using mean or median for all numerical columns."""
        num_cols = self.df.select_dtypes(include=[np.number]).columns
        for col in num_cols:
            self.df[col].fillna(self.df.groupby(group_by)[col].transform(strategy), inplace=True)
    # 4. Remplir par la valeur la plus fréquente (catégories)
    def impute_categorical(self, strategy='most_frequent', fill_value=None):
        """Impute missing values in all categorical columns."""
        cat_cols = self.df.select_dtypes(exclude=[np.number]).columns
        for col in cat_cols:
            if strategy == 'constant' and fill_value is not None:
                self.df[col].fillna(fill_value, inplace=True)
            else:
                imputer = SimpleImputer(strategy=strategy)
                self.df[col] = imputer.fit_transform(self.df[[col]]).ravel()
    # 5. Remplissage en avant et en arrière
    def forward_fill(self):
        """Forward fill missing values for all columns."""
        self.df.fillna(method='ffill', inplace=True)
    
    def backward_fill(self):
        """Backward fill missing values for all columns."""
        self.df.fillna(method='bfill', inplace=True)
    # 6. Interpolation (utile pour les séries temporelles)
    def interpolate(self, method='linear'):
        """Interpolate missing values for all numerical columns."""
        num_cols = self.df.select_dtypes(include=[np.number]).columns
        for col in num_cols:
            self.df[col].interpolate(method=method, inplace=True)
    # using KNN for all numerical columns
    def knn_imputation(self, n_neighbors=5):
        """Impute missing values using KNN for all numerical columns."""
        imputer = KNNImputer(n_neighbors=n_neighbors)
        self.df[:] = imputer.fit_transform(self.df)
    # using Iterative Imputer (MICE)
    def iterative_imputation(self):
        """Impute missing values using Iterative Imputer (MICE) for all numerical columns."""
        imputer = IterativeImputer()
        self.df[:] = imputer.fit_transform(self.df)
    
    def get_dataframe(self):
        """Return the processed DataFrame."""
        return self.df


In [56]:
# Importer la classe
handler = MissingDataHandler(df)

# Supprimer les colonnes avec plus de 50% de valeurs manquantes
handler.drop_column(threshold=0.5)

# Imputation des valeurs manquantes
handler.impute_mean_median(strategy='median')  # Impute par la moyenne pour les colonnes numériques
handler.impute_categorical(strategy='most_frequent')  # Impute par la valeur la plus fréquente pour les catégoriques

# Utiliser forward fill ou backward fill si nécessaire
handler.forward_fill()

# Récupérer le DataFrame traité
cleaned_df = handler.get_dataframe()

print("\nDataFrame après traitement :")
print(cleaned_df)


DataFrame après traitement :
                              MS_SubClass                 MS_Zoning  \
0     One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
1     One_Story_1946_and_Newer_All_Styles  Residential_High_Density   
2     One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
3     One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
4                Two_Story_1946_and_Newer   Residential_Low_Density   
...                                   ...                       ...   
2925                  Split_or_Multilevel   Residential_Low_Density   
2926  One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
2927                          Split_Foyer   Residential_Low_Density   
2928  One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
2929             Two_Story_1946_and_Newer   Residential_Low_Density   

      Lot_Frontage  Lot_Area Street            Alley           Lot_Shape  \
0            141.0   31770.0   Pave  No_A

C:\Users\Client\AppData\Local\Temp\ipykernel_7236\2934517159.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df.fillna(method='ffill', inplace=True)


In [57]:
def na(df, percent = True):
    srs = df.isna().sum()[df.isna().sum() > 0]
    srs = srs.sort_values(ascending=False)
    if percent:
        print('% of NaNs in df:')
        return srs / df.shape[0]
    else:
        print('# of NaNs in df:')
        return srs

na(df, False)

# of NaNs in df:


Series([], dtype: int64)

#### Removing duplicates rows

In [58]:
def remove_duplicates(df, subset=None, keep='first', inplace=False):
    """
    Removes duplicate rows from the DataFrame and provides a summary of duplicates.
    Parameters:
        df (pd.DataFrame): The DataFrame from which to remove duplicates.
        subset (list): List of columns to consider for duplicate checking. 
                    If None, checks all columns.
        keep (str): Which duplicates to keep. Options:
            - 'first': Keep the first occurrence (default).
            - 'last': Keep the last occurrence.
            - 'none': Drop all duplicates.
        inplace (bool): If True, modifies the original DataFrame. 
                        If False, returns a new DataFrame.
    Returns:
        pd.DataFrame: DataFrame with duplicates removed (if inplace=False).
    """
    if keep not in ['first', 'last', 'none']:
        raise ValueError("keep must be one of 'first', 'last', or 'none'.")
    # Count duplicates before removal
    total_rows = len(df)
    duplicate_rows = df.duplicated(subset=subset, keep=False).sum()
    percentage_duplicates = (duplicate_rows / total_rows) * 100
    print(f"Total Rows: {total_rows}")
    print(f"Duplicate Rows: {duplicate_rows} ({percentage_duplicates:.2f}%)")
    if duplicate_rows == 0:
        print("No duplicates found. No rows removed.")
        return df if not inplace else None
    # Handle duplicate removal    if keep == 'none':
        # Drop all duplicates and keep only unique rows
        duplicated_mask = df.duplicated(subset=subset, keep=False)
        result = df[~duplicated_mask]
    else:
        # Use pandas built-in drop_duplicates
        result = df.drop_duplicates(subset=subset, keep=keep)
    # Count duplicates after removal
    remaining_rows = len(result)
    rows_removed = total_rows - remaining_rows
    print(f"Rows Removed: {rows_removed}")
    print(f"Remaining Rows: {remaining_rows}")
    if rows_removed > 0:
        print("Duplicates successfully removed.")
    else:
        print("No duplicates were removed.")
    if inplace:
        df.drop_duplicates(subset=subset, keep=keep, inplace=True)
    else:
        return result

In [59]:
dm=remove_duplicates(df)

Total Rows: 2930
Duplicate Rows: 0 (0.00%)
No duplicates found. No rows removed.


#### outliers

In [60]:
def handle_outliers(df, numerical_columns=None, method='IQR', verbose=True):
    """
    Handles outliers in continuous numerical variables using the specified method.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        numerical_columns (list): List of numerical columns to check for outliers. If None, auto-selects all numerical columns.
        method (str): Method to handle outliers ('IQR' or 'Z-Score'). Default is 'IQR'.
        verbose (bool): If True, displays outlier statistics.
        
    Returns:
        pd.DataFrame: DataFrame with outliers handled.
    """
    # Select numerical columns if not provided
    if numerical_columns is None:
        numerical_columns = df.select_dtypes(include=['number']).columns
    # Iterate over each numerical column
    for col in numerical_columns:
        if col in df.columns:
            if method == 'IQR':
                # Calculate Q1, Q3, and IQR
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                # Count and handle outliers
                outliers = ((df[col] < lower_bound) | (df[col] > upper_bound))
                num_outliers = outliers.sum()
                
                # Option 1: Capping
                df.loc[df[col] < lower_bound, col] = lower_bound
                df.loc[df[col] > upper_bound, col] = upper_bound
                
                if verbose:
                    print(f"[INFO] Outliers handled in '{col}' using IQR:")
                    print(f"         - Number of Outliers: {num_outliers}")
                    print(f"         - Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
            elif method == 'Z-Score':
                # Alternative method (if needed)
                mean = df[col].mean()
                std_dev = df[col].std()
                threshold = 3  # Standard deviation threshold
                
                # Count and handle outliers
                outliers = ((df[col] < (mean - threshold * std_dev)) | (df[col] > (mean + threshold * std_dev)))
                num_outliers = outliers.sum()
                
                # Option 1: Capping
                lower_bound = mean - threshold * std_dev
                upper_bound = mean + threshold * std_dev
                df.loc[df[col] < lower_bound, col] = lower_bound
                df.loc[df[col] > upper_bound, col] = upper_bound
                
                if verbose:
                    print(f"[INFO] Outliers handled in '{col}' using Z-Score:")
                    print(f"         - Number of Outliers: {num_outliers}")
                    print(f"         - Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    
    return df

In [61]:
dc=handle_outliers(dm, numerical_columns=None, method='IQR', verbose=True)

[INFO] Outliers handled in 'Lot_Frontage' using IQR:
         - Number of Outliers: 31
         - Lower Bound: -9.5, Upper Bound: 130.5
[INFO] Outliers handled in 'Lot_Area' using IQR:
         - Number of Outliers: 127
         - Lower Bound: 1267.75, Upper Bound: 17727.75
[INFO] Outliers handled in 'Year_Built' using IQR:
         - Number of Outliers: 9
         - Lower Bound: 1883.5, Upper Bound: 2071.5
[INFO] Outliers handled in 'Year_Remod_Add' using IQR:
         - Number of Outliers: 0
         - Lower Bound: 1906.5, Upper Bound: 2062.5
[INFO] Outliers handled in 'Mas_Vnr_Area' using IQR:
         - Number of Outliers: 203
         - Lower Bound: -244.125, Upper Bound: 406.875
[INFO] Outliers handled in 'BsmtFin_SF_1' using IQR:
         - Number of Outliers: 0
         - Lower Bound: -3.0, Upper Bound: 13.0
[INFO] Outliers handled in 'BsmtFin_SF_2' using IQR:
         - Number of Outliers: 351
         - Lower Bound: 0.0, Upper Bound: 0.0
[INFO] Outliers handled in 'Bsmt_Unf_S

[INFO] Outliers handled in 'Gr_Liv_Area' using IQR:
         - Number of Outliers: 75
         - Lower Bound: 200.875, Upper Bound: 2667.875
[INFO] Outliers handled in 'Bsmt_Full_Bath' using IQR:
         - Number of Outliers: 2
         - Lower Bound: -1.5, Upper Bound: 2.5
[INFO] Outliers handled in 'Bsmt_Half_Bath' using IQR:
         - Number of Outliers: 175
         - Lower Bound: 0.0, Upper Bound: 0.0
[INFO] Outliers handled in 'Full_Bath' using IQR:
         - Number of Outliers: 4
         - Lower Bound: -0.5, Upper Bound: 3.5
[INFO] Outliers handled in 'Half_Bath' using IQR:
         - Number of Outliers: 0
         - Lower Bound: -1.5, Upper Bound: 2.5
[INFO] Outliers handled in 'Bedroom_AbvGr' using IQR:
         - Number of Outliers: 78
         - Lower Bound: 0.5, Upper Bound: 4.5
[INFO] Outliers handled in 'Kitchen_AbvGr' using IQR:
         - Number of Outliers: 134
         - Lower Bound: 1.0, Upper Bound: 1.0
[INFO] Outliers handled in 'TotRms_AbvGrd' using IQR:
     

#### Data type conversions

In [62]:
def get_object_columns(df):
    """
    Cette fonction retourne la liste des colonnes de type 'object' dans un DataFrame.
    
    Paramètres :
    df (pd.DataFrame) : Le DataFrame à analyser.
    
    Retour :
    list : Liste des noms de colonnes de type 'object'.
    """
    return df.select_dtypes(include='object').columns.tolist()

# Exemple d'utilisation
# df = pd.read_csv('ton_fichier.csv')
object_columns = get_object_columns(df)
print(len(object_columns), "\n", object_columns)


38 
 ['MS_SubClass', 'MS_Zoning', 'Street', 'Alley', 'Lot_Shape', 'Land_Contour', 'Utilities', 'Lot_Config', 'Land_Slope', 'Neighborhood', 'Condition_1', 'Condition_2', 'Bldg_Type', 'House_Style', 'Overall_Cond', 'Roof_Style', 'Roof_Matl', 'Exterior_1st', 'Exterior_2nd', 'Exter_Cond', 'Foundation', 'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating', 'Heating_QC', 'Central_Air', 'Electrical', 'Functional', 'Garage_Type', 'Garage_Finish', 'Garage_Cond', 'Paved_Drive', 'Pool_QC', 'Fence', 'Sale_Type', 'Sale_Condition']


In [63]:
def label_encode_columns(df, columns=object_columns, verbose=True):
    """
    Performs Label Encoding on specified categorical columns.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to encode.
        columns (list): List of column names to be label encoded.
        verbose (bool): If True, displays the label mappings.
        
    Returns:
        pd.DataFrame: DataFrame with encoded columns.
    """
    # Loop through the specified columns and encode them
    for col in columns:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            
            # Display label mapping
            if verbose:
                mapping = dict(zip(le.classes_, le.transform(le.classes_)))
                print(f"\n[INFO] Label Encoding for '{col}': {mapping}")
        else:
            print(f"[WARNING] Column '{col}' not found in DataFrame.")
    
    return df

In [64]:
dc=label_encode_columns(dc, columns=object_columns, verbose=True)


[INFO] Label Encoding for 'MS_SubClass': {'Duplex_All_Styles_and_Ages': 0, 'One_Story_1945_and_Older': 1, 'One_Story_1946_and_Newer_All_Styles': 2, 'One_Story_PUD_1946_and_Newer': 3, 'One_Story_with_Finished_Attic_All_Ages': 4, 'One_and_Half_Story_Finished_All_Ages': 5, 'One_and_Half_Story_PUD_All_Ages': 6, 'One_and_Half_Story_Unfinished_All_Ages': 7, 'PUD_Multilevel_Split_Level_Foyer': 8, 'Split_Foyer': 9, 'Split_or_Multilevel': 10, 'Two_Family_conversion_All_Styles_and_Ages': 11, 'Two_Story_1945_and_Older': 12, 'Two_Story_1946_and_Newer': 13, 'Two_Story_PUD_1946_and_Newer': 14, 'Two_and_Half_Story_All_Ages': 15}

[INFO] Label Encoding for 'MS_Zoning': {'A_agr': 0, 'C_all': 1, 'Floating_Village_Residential': 2, 'I_all': 3, 'Residential_High_Density': 4, 'Residential_Low_Density': 5, 'Residential_Medium_Density': 6}

[INFO] Label Encoding for 'Street': {'Grvl': 0, 'Pave': 1}

[INFO] Label Encoding for 'Alley': {'Gravel': 0, 'No_Alley_Access': 1, 'Paved': 2}

[INFO] Label Encoding for 

### feature eng

In [65]:
mean_sales= dc["Sale_Price"].mean()
max_sales= dc["Sale_Price"].max()
min_sales= dc["Sale_Price"].min()
std_sales = dc["Sale_Price"].std()
dc["Mean_Sale_Price"] = mean_sales
dc["Max_Sale_Price"] = max_sales
dc["Min_Sale_Price"] = min_sales
dc["Std_Sale_Price"] = std_sales

In [66]:
dc["Total_Bathrooms"] = dc["Bsmt_Full_Bath"] + dc["Full_Bath"] + 0.5 * (dc["Bsmt_Half_Bath"] + dc["Half_Bath"])
dc["Total_Living_Area"] = dc["Gr_Liv_Area"] + dc["Total_Bsmt_SF"]
dc["Land_Living_Ratio"] = dc["Lot_Area"] / dc["Total_Living_Area"]
dc["Room_Density"] = dc["Total_Living_Area"] / dc["TotRms_AbvGrd"]
dc["Has_Garage"] = dc["Garage_Area"].apply(lambda x: 1 if x > 0 else 0)
dc["Garage_Size_Per_Car"] = dc["Garage_Area"] / (dc["Garage_Cars"] + 1)
dc["Total_Porch_Area"] = dc["Wood_Deck_SF"] + dc["Open_Porch_SF"] + dc["Enclosed_Porch"] + dc["Screen_Porch"] + dc["Three_season_porch"]
expensive_neighborhoods = dc.groupby("Neighborhood")["Sale_Price"].median().sort_values(ascending=False).head(5).index
dc["High_Value_Neighborhood"] = dc["Neighborhood"].apply(lambda x: 1 if x in expensive_neighborhoods else 0)
dc["Has_Paved_Drive"] = dc["Paved_Drive"].map({"Y": 1, "N": 0})
dc["Lot_Area_Log"] = np.log1p(dc["Lot_Area"])
dc["Sale_Price_Log"] = np.log1p(dc["Sale_Price"])
dc["Gr_Liv_Area_Log"] = np.log1p(dc["Gr_Liv_Area"])
dc["House_Age"] = dc["Year_Sold"] - dc["Year_Built"]

In [67]:
cos_price_sales= np.cos(dc["Sale_Price_Log"])
sin_price_sale= np.sin(dc["Sale_Price_Log"])
dc["cos_price_sales"]=cos_price_sales
dc["sin_price_sale"]=sin_price_sale

In [68]:
cols_used = [
    "Bsmt_Full_Bath", "Full_Bath", "Bsmt_Half_Bath", "Half_Bath",  # Utilisés dans Total_Bathrooms
    "Gr_Liv_Area", "Total_Bsmt_SF",  # Utilisés dans Total_Living_Area
    "Lot_Area", "Total_Living_Area",  # Utilisés dans Land_Living_Ratio
    "Total_Living_Area", "TotRms_AbvGrd",  # Utilisés dans Room_Density
    "Garage_Area",  # Utilisé dans Has_Garage et Garage_Size_Per_Car
    "Garage_Cars",  # Utilisé dans Garage_Size_Per_Car
    "Wood_Deck_SF", "Open_Porch_SF", "Enclosed_Porch", "Screen_Porch", "Three_season_porch",  # Utilisés dans Total_Porch_Area
    "Neighborhood",  # Utilisé dans High_Value_Neighborhood
    "Paved_Drive",  # Utilisé dans Has_Paved_Drive
    "Lot_Area",  # Transformé avec Lot_Area_Log
    "Gr_Liv_Area",  # Transformé avec Gr_Liv_Area_Log
    "Year_Built", "Year_Sold"  # Utilisés dans House_Age
]
cols_useless = ['Has_Paved_Drive'
]
cols_to_drop = cols_used + cols_useless
dc.drop(columns=cols_to_drop, inplace=True, errors='ignore')

### model

In [69]:
from sklearn.model_selection import train_test_split
x= dc.drop(columns='Sale_Price_Log')
y= dc['Sale_Price_Log']

In [70]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=20)

In [71]:
# Initialize the scaler
scaler = StandardScaler()

# Fit and transform only the selected columns
x_train = scaler.fit_transform(x_train)
x_test  = scaler.fit_transform(x_test)
#x_t[columns_to_scale] = scaler.fit_transform(x_t[columns_to_scale])
#x_te[columns_to_scale] = scaler.fit_transform(x_te[columns_to_scale])

In [72]:
dc.isna().sum()

MS_SubClass        0
MS_Zoning          0
Lot_Frontage       0
Street             0
Alley              0
                  ..
Sale_Price_Log     0
Gr_Liv_Area_Log    0
House_Age          0
cos_price_sales    0
sin_price_sale     0
Length: 70, dtype: int64

#### linear regression 

In [73]:
# Initialize Linear Regression model
lr = LinearRegression()

# Timing the training process
training_start = time.perf_counter()
model = lr.fit(x_train, y_train)
training_end = time.perf_counter()

# Timing the prediction process
prediction_start = time.perf_counter()
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)
prediction_end = time.perf_counter()

# Calculate KPIs
# Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

# Mean Absolute Error (MAE)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test , y_pred_test)

# R-squared score
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

# Time metrics
train_time = training_end - training_start
prediction_time = prediction_end - prediction_start


In [74]:
# Output metrics
print("Linear Regression Model Performance:")
print("\nTraining Set:")
print(f"  - RMSE: {rmse_train:.4f}")
print(f"  - MAE: {mae_train:.4f}")
print(f"  - R^2 Score: {r2_train:.4f}")
print(f"  - Training Time: {train_time:.4f} seconds")

print("\nTesting Set:")
print(f"  - RMSE: {rmse_test:.4f}")
print(f"  - MAE: {mae_test:.4f}")
print(f"  - R^2 Score: {r2_test:.4f}")
print(f"  - Prediction Time: {prediction_time:.5f} seconds")

Linear Regression Model Performance:

Training Set:
  - RMSE: 0.0030
  - MAE: 0.0020
  - R^2 Score: 0.9999
  - Training Time: 1.0843 seconds

Testing Set:
  - RMSE: 0.0571
  - MAE: 0.0219
  - R^2 Score: 0.9810
  - Prediction Time: 0.00565 seconds


In [75]:
print(np.exp(y_test.mean()))
print(np.exp(y_test.std()))

162842.63499267888
1.5135427826952261


#### Ridge and Lasso regression

cross validation 

In [76]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


# Define models and parameter grids for GridSearchCV
ridge = Ridge()
lasso = Lasso()

ridge_params = {'alpha': [0.01, 0.1, 1, 10, 100]}
lasso_params = {'alpha': [0.01, 0.1, 1, 10, 100,200,300,400,500]}
ridge_grid = GridSearchCV(estimator=ridge, param_grid=ridge_params, 
                        cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
# Lasso Regression Hypertuning
lasso_grid = GridSearchCV(estimator=lasso, param_grid=lasso_params, 
                        cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

Ridge and Lasso 

In [77]:
ridge_grid.fit(x_train, y_train)
best_ridge = ridge_grid.best_estimator_
print("\nBest Ridge Model:", ridge_grid.best_params_)


lasso_grid.fit(x_train, y_train)
best_lasso = lasso_grid.best_estimator_
print("\nBest Lasso Model:", lasso_grid.best_params_)


Best Ridge Model: {'alpha': 0.01}

Best Lasso Model: {'alpha': 0.01}


In [78]:
# Evaluate the best models
best_models = {'Ridge': best_ridge, 'Lasso': best_lasso}

for model_name, model in best_models.items():
    print(f"\nEvaluating {model_name} Model:")

    # Timing the training process
    training_start = time.perf_counter()
    model.fit(x_train, y_train)
    training_end = time.perf_counter()
    
    # Timing the prediction process
    prediction_start = time.perf_counter()
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    prediction_end = time.perf_counter()

    # Calculate KPIs
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)

    # Time metrics
    train_time = training_end - training_start
    prediction_time = prediction_end - prediction_start
    # Output metrics
    print("\nTraining Set:")
    print(f"  - RMSE: {rmse_train:.4f}")
    print(f"  - MAE: {mae_train:.4f}")
    print(f"  - R^2 Score: {r2_train:.4f}")
    print(f"  - Training Time: {train_time:.4f} seconds")

    print("\nTesting Set:")
    print(f"  - RMSE: {rmse_test:.4f}")
    print(f"  - MAE: {mae_test:.4f}")
    print(f"  - R^2 Score: {r2_test:.4f}")
    print(f"  - Prediction Time: {prediction_time:.5f} seconds")


Evaluating Ridge Model:

Training Set:
  - RMSE: 0.0030
  - MAE: 0.0020
  - R^2 Score: 0.9999
  - Training Time: 0.0067 seconds

Testing Set:
  - RMSE: 0.0571
  - MAE: 0.0219
  - R^2 Score: 0.9810
  - Prediction Time: 0.00139 seconds

Evaluating Lasso Model:

Training Set:
  - RMSE: 0.0110
  - MAE: 0.0085
  - R^2 Score: 0.9992
  - Training Time: 0.0475 seconds

Testing Set:
  - RMSE: 0.0679
  - MAE: 0.0293
  - R^2 Score: 0.9731
  - Prediction Time: 0.00181 seconds


#### modele ElasticNet

In [82]:
# Définition et entraînement du modèle ElasticNet
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

# Définir les hyperparamètres à tester
param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Force de régularisation
    'l1_ratio': [0.2, 0.5, 0.8]  # Équilibre entre L1 (Lasso) et L2 (Ridge)
}

elasticnet = ElasticNet()
grid_search = GridSearchCV(elasticnet, param_grid, cv=5, scoring='r2')
grid_search.fit(x_train, y_train)

# Meilleur modèle après validation croisée
best_elasticnet = grid_search.best_estimator_


In [83]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import numpy as np
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Meilleurs modèles après validation croisée
best_models = {'Ridge': best_ridge, 'Lasso': best_lasso, 'ElasticNet': best_elasticnet}

for model_name, model in best_models.items():
    print(f"\nEvaluating {model_name} Model:")

    # Timing the training process
    training_start = time.perf_counter()
    model.fit(x_train, y_train)
    training_end = time.perf_counter()
    
    # Timing the prediction process
    prediction_start = time.perf_counter()
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    prediction_end = time.perf_counter()

    # Calculate KPIs
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)

    # Time metrics
    train_time = training_end - training_start
    prediction_time = prediction_end - prediction_start
    
    # Output metrics
    print("\nTraining Set:")
    print(f"  - RMSE: {rmse_train:.4f}")
    print(f"  - MAE: {mae_train:.4f}")
    print(f"  - R^2 Score: {r2_train:.4f}")
    print(f"  - Training Time: {train_time:.4f} seconds")

    print("\nTesting Set:")
    print(f"  - RMSE: {rmse_test:.4f}")
    print(f"  - MAE: {mae_test:.4f}")
    print(f"  - R^2 Score: {r2_test:.4f}")
    print(f"  - Prediction Time: {prediction_time:.5f} seconds")



Evaluating Ridge Model:

Training Set:
  - RMSE: 0.0030
  - MAE: 0.0020
  - R^2 Score: 0.9999
  - Training Time: 0.0791 seconds

Testing Set:
  - RMSE: 0.0571
  - MAE: 0.0219
  - R^2 Score: 0.9810
  - Prediction Time: 0.00552 seconds

Evaluating Lasso Model:

Training Set:
  - RMSE: 0.0110
  - MAE: 0.0085
  - R^2 Score: 0.9992
  - Training Time: 0.0306 seconds

Testing Set:
  - RMSE: 0.0679
  - MAE: 0.0293
  - R^2 Score: 0.9731
  - Prediction Time: 0.00113 seconds

Evaluating ElasticNet Model:

Training Set:
  - RMSE: 0.0330
  - MAE: 0.0220
  - R^2 Score: 0.9925
  - Training Time: 0.0083 seconds

Testing Set:
  - RMSE: 0.1046
  - MAE: 0.0418
  - R^2 Score: 0.9362
  - Prediction Time: 0.00136 seconds
